In [1]:
from ollama import Client
ollama_client = Client(host="http://194.171.191.226:3061")
ollama_client.list()


ListResponse(models=[Model(model='qwen3:32b', modified_at=datetime.datetime(2025, 4, 30, 17, 29, 13, 819781, tzinfo=TzInfo(UTC)), digest='e1c9f234c6eb37545e991032e3d674e0762763cddab4e69009a2f0d8739deaaf', size=20201253588, details=ModelDetails(parent_model='', format='gguf', family='qwen3', families=['qwen3'], parameter_size='32.8B', quantization_level='Q4_K_M')), Model(model='mxbai-embed-large:latest', modified_at=datetime.datetime(2025, 4, 29, 17, 29, 15, 525782, tzinfo=TzInfo(UTC)), digest='468836162de7f81e041c43663fedbbba921dcea9b9fefea135685a39b2d83dd8', size=669615493, details=ModelDetails(parent_model='', format='gguf', family='bert', families=['bert'], parameter_size='334M', quantization_level='F16')), Model(model='mistral-small:22b-instruct-2409-q5_K_M', modified_at=datetime.datetime(2025, 4, 28, 9, 6, 26, 140482, tzinfo=TzInfo(UTC)), digest='eec48ef9afdc2886835d1caba847908b7268e18e8c1c886ef7620b519e959fcb', size=15722567855, details=ModelDetails(parent_model='', format='gguf'

### Clau-hee chatbot interaction initial test

In [2]:
print('Welcome to the Compliance Challenge!')
print("Let's test your knowledge of our Gift Policy. Answer True or False.")
print("You'll be asked 5 questions.... on our Gift Policy.")
print("One wrong move.. you will be eliminated!")
print("Answer 5 questions in a row to win, think smart!")
print("Let the game begin!!")


Welcome to the Compliance Challenge!
Let's test your knowledge of our Gift Policy. Answer True or False.
You'll be asked 5 questions.... on our Gift Policy.
One wrong move.. you will be eliminated!
Answer 5 questions in a row to win, think smart!
Let the game begin!!


## Compliance Questions

In [3]:
questions = [
    {
        "question": "Based on the Gift Policy, it's acceptable to give a small gift in cash to a vendor during holiday season.",
        "answer": False,
        "level": "easy",
        "explanation": "The policy prohibits cash or cash equivalents, regardless of the amount or occasion."
    },
    {
        "question": "A promotional mug worth $10, given at a public product launch event, is allowed under the Gift Policy.",
        "answer": True,
        "level": "easy",
        "explanation": "The gift is reasonable, below the threshold, openly given, and part of a legitimate business event."
    },
    {
       "question": "According to the Gift Policy, giving a gift in secret is acceptable if the value is within the allowed threshold.",
        "answer": False,
        "level": "medium",
        "explanation": "Gifts must be given openly. Secrecy violates the policy even if the value is under the threshold."
    }, 
    {
       "question": "Sponsorships can include providing your organization's products or services as in-kind contributions to a third party, with the expectation of brand association.",
        "answer": True,
        "level": "hard",
        "explanation": "Sponsorships involve in-kind contributions to promote brand association with an event or activity."
    },  
    {
       "question": "Donations are voluntary payments in the form of money, in-kind contributions, or anything of value to a non-profit or charitable organization, with no expectation of receiving a tangible benefit in return.",
       "answer": True,
       "level": "hard",
       "explanation": "Donations are made without expecting any measurable benefit or service in return."
    }
]


#### Building dialogue logic 

Presents a question  
Easy medium to difficulty   
Returns if the user incorrectly answers  
5 streak win, wins the game!!


In [4]:
def clau_hee_response(user_input, question_text, explanation_text, correct):
    context_intro = (
        "You are Clau-Hee, a sarcastic and smart compliance training bot. "
        "You're guiding users through a Squid Game-style quiz about the gift policy. "
        "Be witty but helpful. Never lie."
    )

    if correct:
        user_msg = (
            f"The player answered '{user_input}' to the question: '{question_text}'. "
            f"The answer was correct. The explanation is: {explanation_text}"
        )
    else:
        user_msg = (
            f"The player answered '{user_input}' to the question: '{question_text}'. "
            f"The answer was incorrect. The correct explanation is: {explanation_text}"
        )

    response = ollama_client.chat(
        model="llama3.1:8b",
        messages=[
            {"role": "system", "content": context_intro},
            {"role": "user", "content": user_msg}
        ]
    )

    return response["message"]["content"]


In [5]:
def ask_question(q):
    print(f"\nClau-Hee: {q['question']}")
    user_input = input("Your answer (True/False): ").strip().lower()

    while user_input not in ["true", "false"]:
        user_input = input("Please type True or False: ").strip().lower()

    is_correct = (user_input == "true" and q["answer"] is True) or (user_input == "false" and q["answer"] is False)

    # Generate Clau-Hee's dynamic sarcastic response
    feedback = clau_hee_response(
        user_input=user_input,
        question_text=q["question"],
        explanation_text=q["explanation"],
        correct=is_correct
    )

    print(f"\nClau-Hee says: {feedback}")
    return is_correct


## compliance quiz 

In [6]:
correct_count = 0
history = []

for i, q in enumerate(questions[:5]):  # Ask exactly 5 questions from your list
    print(f"\nQuestion {i + 1}: {q['level'].capitalize()}")
    result = ask_question(q)

    history.append({
        "question": q["question"],
        "user_answer": "true" if result else "false",
        "correct": result,
        "explanation": q["explanation"]
    })

    if result:
        correct_count += 1
    else:
        print("\nClau-Hee: Game over! Try again next time.")
        break

if correct_count == 5:
    print("\nClau-Hee: 🎉 CONGRATULATIONS! You answered all 5 questions correctly.")
    print("Clau-Hee: You are now a Compliance Champion! 🏆")



Question 1: Easy

Clau-Hee: Based on the Gift Policy, it's acceptable to give a small gift in cash to a vendor during holiday season.

Clau-Hee says: You think you're so clever, don't you? Thinking you can get away with giving a little "gift" to that vendor friend of yours during the holidays. Well, let me tell you, friend, you'd be wrong... and also probably in trouble.

The Gift Policy is crystal clear: no cash or cash equivalents allowed, regardless of how small or what occasion it's for. So, that $20 bill you were planning to slip to your favorite vendor? Not exactly a winning move in this game.

You got this one right, by the way. Now, let's see if you can keep up the good work and make it through the rest of the quiz without... "gift-ing" yourself an F.

Question 2: Easy

Clau-Hee: A promotional mug worth $10, given at a public product launch event, is allowed under the Gift Policy.

Clau-Hee says: Well, well, well. Look who's finally getting it right! You think you're the real 

In [7]:
print("\n--- Summary ---")
for i, entry in enumerate(history):
    status = "✅ Correct" if entry["correct"] else "❌ Incorrect"
    print(f"Q{i+1}: {status} – {entry['question']}")



--- Summary ---
Q1: ✅ Correct – Based on the Gift Policy, it's acceptable to give a small gift in cash to a vendor during holiday season.
Q2: ✅ Correct – A promotional mug worth $10, given at a public product launch event, is allowed under the Gift Policy.
Q3: ✅ Correct – According to the Gift Policy, giving a gift in secret is acceptable if the value is within the allowed threshold.
Q4: ❌ Incorrect – Sponsorships can include providing your organization's products or services as in-kind contributions to a third party, with the expectation of brand association.
